# EV State-Space Model

This  notebook is used to reproduce the State Space Model of aggregateed EV for frequency regulation.

Running on Jinning's local machine, in the env "ev".

Working notes:

02/06/2022: the EV data generation is completed. It looks weired, we may need update it later on.


## EV data

### Define parameters

The EV parameters are defined as two types. Type I follows uniform distribution, which is stored in a Dict `ev_param`. Type II follows normal distribution, which is stored in a pd.DataFrame `ev_pdf`.

The data are cited from:

M. Wang et al., "State Space Model of Aggregated Electric Vehicles for Frequency Regulation," in IEEE Transactions on Smart Grid, vol. 11, no. 2, pp. 981-994, March 2020, doi: 10.1109/TSG.2019.2929052.

In [1]:
import itertools
import random
from tqdm import tqdm
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
from hmmlearn import hmm

import logging
logger = logging.getLogger(__name__)


def find_x(x, soc_intv):
    out = -1
    for idx in soc_intv.keys():
        if x > soc_intv[idx][0] and x <= soc_intv[idx][1]:
            out = idx
    return out


def update_xl(inl_input):
    """
    Update x series.
    columns:
    ['socx', 'xl', 'u', 'u0', 'ctrl']
      0,   1,    2,    3,     4
    """
    inl = inl_input.copy()
    x = inl[0]
    dc = inl[1].copy()
    u = inl[2]
    u0 = inl[3]
    state = inl[4]
    # --- Continuous online ---
    if u*u0 == 1:
        if len(dc[0]) == 0:
            # print("Online: init")
            dc[0] = [state]
            dc[1] = [x]
        else:
            # print("Online: not init")
            dc[0].append(state)
            dc[1].append(x)
    # --- offline -> online ---
    elif (1-u0)*u == 1:
        if len(dc[0]) == 0:
            # print("Offline -> online: init")
            dc[0] = [state]
            dc[1] = [x]
        else:
            # print("Offline -> online: not init")
            dc[0].append([state])
            dc[1].append([x])
    return dc


def safe_div(x, y):
    if y == 0:
        return 0
    else:
        return x/y


class ev_ssm():
    """
    EV State Space Model.

    EV parameters:
    Pc, Pd, nc, nd, En follow uniform distribution.
    soci, socd, ts, tf follows normal distribution.

    Attributes
    ----------
    xtab: pandas.DataFrame
        EV states table, only online EVs are counted.
    ne: int
        Number of online EVs
    N: int
        Number of total EVs
    ev: pandas.DataFrame
        EV dataset
        u: online status
        Pc/Pd: charging/discharging power (kW)
        nc/nd: charging/discharging efficiency
        En: capacity (kWh)
        soci/socd: initial/demand SOC (unit: %)
        ts/tf: start/finish time (unit: 24H)
        socx: SOC interval
    tnow: float
        current time (unit: 24H)
    Ns: int
        Number of SoC intervals

    Notes
    -----
    ev_ufparam:
        Pl: rated charging/discharging power (kW) lower bound
        Pu: rated charging/discharging power (kW) upper bound
        nl: charging/discharging efficiency lower bound
        nu: charging/discharging efficiency upper bound
        Enl: Battery capacity (kWh) lower bound
        Enu: Battery capacity (kWh) upper bound
        socl: Minimum SoC value
        socu: Maximum SoC value
    ev_nfparam:
        soci: initial SoC
        socd: demanded SoC
        ts1: start charging time, [24H]
        ts2: start charging time, [24H]
        tf1: finish charging time, [24H]
        tf2: finish charging time, [24H]
    """

    def find_socx(self):
        self.ev['socx'] = self.ev['soc'].apply(lambda x: find_x(x, self.soc_intv))

    def report(self):
        """
        Report EVA.
        """
        # --- EV summary info ---
        self.En = self.ev.En.sum()/1e3
        self.wEn = np.sum(self.ev.u * self.ev.En)/1e3
        # --- report info ---
        msg1 = f"{self.name}:\n"
        msg_time = f'tnow={np.round(self.tnow, 4)} [24H]\n'
        msg_ev = f"{self.N} EVs, {self.ne} online, Total En={self.En.round(2)} MWh, SoC intervals: {len(self.soc_intv)}\n"
        msg_soc = f"Online En={self.wEn.round(2)} MWh, mean SoC={100*self.ev.soc.mean().round(2)}%"
        logger.warning(msg1 + msg_time + msg_ev + msg_soc)

    def g_tnow(self, tnow):
        """
        Update time and time series.
        """
        self.ts.append(tnow)
        tnow = tnow if tnow < 24 else tnow-24
        return tnow

    def __init__(self, tnow=0, N=20000, seed=None, name="EVA"):
        """
        Parameters
        ----------
        N: int
            Number of EVs
        seed: int
            Random seed. ``None`` for random.
        """
        # --- 1. init ---
        self.name = name
        self.N = N
        if not seed:
            self.seed = seed
            np.random.seed(self.seed)
        # --- 1a. uniform distribution parameters range ---
        self.ev_ufparam = dict(Ns=20,
                               Pl=5.0, Pu=7.0,
                               nl=0.88, nu=0.95,
                               Enl=20.0, Enu=30.0,
                               socl=0, socu=1)
        #  --- 1b. normal distribution parameters range ---
        self.ev_pdf_name = ['soci', 'socd', 'ts1', 'ts2', 'tf1', 'tf2']
        self.ev_pdf_data = {'mean':     [0.3,    0.8,    -6.5,   17.5,   8.9,    32.9],
                            'var':      [0.05, 0.03, 3.4, 3.4, 3.4, 3.4],
                            'lb':       [0.2, 0.7, 0.0, 5.5, 0.0, 20.9],
                            'ub':       [0.4, 0.9, 5.5, 24.0, 20.9, 24.0],
                            'info':  ['initial SoC', 'demanded SoC',
                                      'start charging time 1', 'start charging time 2',
                                      'finish charging time 1', 'finish charging time 2']}
        self.tnow = tnow
        self.ts = [tnow]
        self.build(tnow=tnow)
        self.report()

        # --- SSM ---
        # --- input: AGC signal ---

        # --- output: estimated FRC ---
        self.prumax = 0
        self.prdmax = 0

    def build(self, tnow):
        """
        Build the ev DataFrame.

        Returns
        -------
        ev: pandas.DataFrame
            EV dataset
        """
        self.socl = self.ev_ufparam['socl']
        self.socu = self.ev_ufparam['socu']

        #  --- 1a. uniform distribution parameters range ---
        unit = self.ev_ufparam['socu']/self.ev_ufparam['Ns']
        self.soc_intv = {}
        decimal = 4
        for i in range(self.ev_ufparam['Ns']):
            intv_single = [np.around(i*unit, decimal), np.around((i+1)*unit, decimal)]
            self.soc_intv[i] = intv_single
        self.Ns = self.ev_ufparam['Ns']
        ev_pdf = pd.DataFrame(data=self.ev_pdf_data, index=self.ev_pdf_name).transpose()
        self.ev_nfparam = ev_pdf.to_dict()

        # --- 1c. generate EV dataset ---
        self.ev = pd.DataFrame()

        #  data from uniform distribution
        cols = ['Pc', 'Pd', 'nc', 'nd', 'En']
        cols_bound = {'Pc':   ['Pl', 'Pu'],
                      'Pd':   ['Pl', 'Pu'],
                      'nc':   ['nl', 'nu'],
                      'nd':   ['nl', 'nu'],
                      'En':    ['Enl', 'Enu']}
        for col in cols:
            idxl = cols_bound[col][0]
            idxh = cols_bound[col][1]
            self.ev[col] = np.random.uniform(
                low=self.ev_ufparam[idxl],
                high=self.ev_ufparam[idxh],
                size=self.N)

        #  data from normal distribution
        # soci, socd
        for col in self.ev_pdf_name:
            self.ev[col] = stats.truncnorm(
                (ev_pdf[col]['lb'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'],
                (ev_pdf[col]['ub'] - ev_pdf[col]['mean']) / ev_pdf[col]['var'],
                loc=ev_pdf[col]['mean'], scale=ev_pdf[col]['var']).rvs(self.N)

        # ts1, ts2, tf1, tf2
        et = self.ev.copy()
        r1 = 0.5  # ratio of t1
        tp1 = self.ev[['ts1', 'tf1']].sample(n=int(et.shape[0]*r1), random_state=2021)
        tp2 = self.ev[['ts2', 'tf2']].sample(n=int(et.shape[0]*(1-r1)), random_state=2021)
        tp = pd.concat([tp1, tp2], axis=0).reset_index(drop=True).fillna(0)
        tp['ts'] = tp['ts1'] + tp['ts2']
        tp['tf'] = tp['tf1'] + tp['tf2']
        check = tp.ts > tp.tf
        row_idx = tp[check].index
        mid = tp.tf.iloc[row_idx].values
        tp.tf.iloc[row_idx] = tp.ts.iloc[row_idx]
        tp.ts.iloc[row_idx] = mid
        check = tp.ts > tp.tf
        self.ev['ts'] = tp['ts']
        self.ev['tf'] = tp['tf']
        self.ev['u'] = 1

        # Initialize delta power
        self.ev['dP'] = 0

        self.states = list(itertools.product([1, 0, -1], self.soc_intv.keys()))
        self.states_str = [str(s[1])+'S'+str(s[0]) for s in self.states]

        # --- update soc interval and online status ---
        # --- ev online status: u0 as u ---
        self.ev['u0'] = 0
        self.g_u()
        self.ev['u0'] = self.ev.u

        # soc is initialzied considering random behavior
        self.ev['soc'] = self.ev[['soci', 'ts', 'Pc', 'nc', 'En', 'tf']].apply(
            lambda x: x[0] + (min(tnow-x[1], x[5]-x[1]))*x[2]*x[3]/x[4] if tnow > x[1] else x[0], axis=1)
        self.ev['soc'] = self.ev['soc'].apply(lambda x: min(x, self.socu))

        # Initiallize control signal
        self.ev['ctrl'] = self.ev['u']  # initially use cahrging signal
        self.g_ctrl(is_charge=False)

        self.find_socx()
        self.g_x()

        ev_cols = ['u', 'u0', 'Pc', 'Pd', 'nc', 'nd', 'En', 'soc',
                   'soci', 'socd', 'socx', 'ts', 'tf', 'ctrl', 'dP']
        self.ev = self.ev[ev_cols]

        # initialize x series
        self.ev['xl'] = [[[], []]] * self.N
        self.ne = self.ev.u.sum()

        self.ssm_basic()
        return True

    def g_u(self):
        """
        Update online status of EV at given time.
        """
        self.ev['u0'] = self.ev.u.astype(int)
        self.ev['u'] = (self.ev.ts <= self.tnow) & (self.ev.tf >= self.tnow)
        self.ev['u'] = self.ev['u'].astype(int)
        self.ne = self.ev.u.sum()
        return True

    def g_x(self):
        """
        Update EV x and SSM x.
        """
        # --- find single EV socx ---
        self.ev['socx'] = self.ev['soc'].apply(lambda x: find_x(x, self.soc_intv))

        # --- find SSM X ---
        states = self.ev[['ctrl', 'socx', 'u']].apply(lambda x: (x[0], x[1]) if x[2] else (-1, -1), axis=1)
        res = dict(states.value_counts())
        self.xtab = pd.DataFrame(columns=range(20), index=[1, 0, -1], data=0)
        for key in res.keys():
            if key[1] > -1:
                self.xtab.loc[key[0], key[1]] = res[key]
        self.xtab.fillna(0, inplace=True)
        return True

    def ssm_basic(self):
        """
        Build SSM B, C, D matrix.
        """
        B1 = -1 * np.eye(self.Ns)
        B2 = np.eye(self.Ns)
        B3 = np.zeros((self.Ns, self.Ns))
        self.B = np.vstack((B1, B2, B3))

        C1 = np.zeros((self.Ns, self.Ns))
        C2 = -1 * np.eye(self.Ns)
        C3 = np.eye(self.Ns)
        self.C = np.vstack((C1, C2, C3))

        # P average
        kde = stats.gaussian_kde(self.ev.Pc)
        Pave = 0  # TODO: consider Pave as an attribute
        step = 0.01
        for Pl in np.arange(self.ev.Pc.min(), self.ev.Pc.max(), step):
            Pave += (kde.integrate_box(Pl, Pl+step)) * (Pl + 0.05 * step)

        D1 = -1 * np.ones((1, self.Ns))
        D2 = np.zeros((1, self.Ns))
        D3 = np.ones((1, self.Ns))
        # TODO: should use online numbers?
        self.D = Pave * self.N * np.hstack((D1, D2, D3))

    def g_A(self):
        """
        Build A matrix.
        """
        # TODO: put the code here, consider the time interval of A

    def run(self, t_step=20, t_end=10, is_charge=False, is_record=True):
        """
        Response of the EV_SSM to the control signal.

        Parameters
        ----------
        t_step: int
            Action time (second).
        tnow: int
            current time (hour with decimals).
        """
        t_step = t_step / 3600
        if t_end - self.tnow < 1e-6:
            logger.warning(f"{self.name}: t_end={t_end} is too close to tnow={self.tnow}")
        else:
            for t in tqdm(np.arange(self.tnow, t_end, t_step), desc=self.name):
                self.tnow = self.g_tnow(t)
                self.g_u()  # update online status
                self.g_ctrl(is_charge=is_charge)  # update control signal

                # --- update soc interval and online status ---
                # charging/discharging power, kW
                self.ev['dP'] = self.ev[['Pc', 'Pd', 'nc', 'nd', 'ctrl', 'u']].apply(
                    lambda x: x[0]*x[2]*x[5] if x[4] >= 0 else -1*x[1]*x[3]*x[5], axis=1)
                # --- update an revise SoC ---
                self.ev['soc'] = self.ev.soc + t_step * self.ev['dP'] / self.ev['En']
                self.ev['soc'] = self.ev['soc'].apply(lambda x: x if x < self.socu else self.socu)
                self.ev['soc'] = self.ev['soc'].apply(lambda x: x if x > self.socl else self.socl)

                # update x
                self.find_socx()
                self.g_x()

                if is_record:
                    self.g_xl()

    def g_A(self):
        """
        Build A matrix after run().
        """
        # --- gather results ---
        seq = []
        states = []
        for item in sse.ev.xl.tolist():
            if len (item[0]) > 0:
                seq.append(item[1])
                states.append(item[0])

        # --- build A with HMM ---
        hmdl = hmm.MultinomialHMM(n_components=3)
        X = np.concatenate(seq).reshape(-1, 1)
        length = [len(item) for item in seq]
        hmdl.fit(X, length)
        self.Tm = hmdl.transmat_
        self.Em = hmdl.emissionprob_

        # --- build A with frequency ---
        Ef = np.zeros((3, sse.Ns))
        for t, s in zip(states, seq):
            for i in range(len(t)):
                Ef[t[i], s[i]] += 1
        rf = Ef.sum(axis=1)
        self.Ef = Ef / rf[:, None]

        Tf = np.zeros((3, 3))
        for state in states:
            for i in range(len(state)-1):
                Tf[state[i+1]][state[i]] += 1
        ra = Tf.sum(axis=1)
        self.Tf = Tf / ra[:,None]

    def g_xl(self):
        """
        Update EV x series.
        """
        self.ev['xl'] = self.ev[['socx', 'xl', 'u', 'u0', 'ctrl']].apply(update_xl, axis=1)

    def g_ctrl(self, is_charge=False):
        """
        Generate the charging signal.
        """
        # --- default signal is charging for all ---
        # TODO: replace with SSM later on
        # --- charging signal ---
        self.ev['ctrl'] = self.ev.u  # charge all online cars
        if is_charge:
            # charged to demanded soc EVs set as idle state
            self.ev['ctrl'] = self.ev[['soc', 'ctrl', 'socd']].apply(
                lambda x: 0 if x[0] >= x[2] else x[1], axis=1)
        else:
            # already max(0.95, socd) EVs are randomly discharging
            self.ev['ctrl'] = self.ev[['ctrl', 'u', 'soc', 'socd']].apply(
                lambda x: np.random.choice([0, -1], p=[0.9, 0.1]) if x[1]*(x[2] > max(x[3], 0.95)) else x[0], axis=1)
        # --- revise control ---
        # low charged EVs are FORCED charging
        self.ev['ctrl'] = self.ev[['soc', 'ctrl']].apply(
            lambda x: 1 if x[0] <= 0.1 else x[1], axis=1)
        # offline EVs set as idle state
        self.ev['ctrl'] = self.ev[['ctrl', 'u']].apply(
            lambda x: x[0]*x[1], axis=1)
        # format
        self.ev['ctrl'] = self.ev['ctrl'].astype(int)
    
    def g_res(self, states_p, mode='hmm'):
        """
        Generate the response of the EV_SSM to the control signal.

        Parameters
        ----------
        states_p: list
            [CS, DS, IS], proportion of states of the initial time step.
        mode: str
            'hmm' stand for Hidden Markov Model, 'freq' stand for frequency estimation.
        """
        # --- charging ---
        res = pd.DataFrame(columns=range(20), index=[1, 0, -1])
        res.loc[1] = [sp[0] * sse.Em[0, socx] for socx in range(20)]
        res.loc[0] = [sp[1] * sse.Em[1, socx] for socx in range(20)]
        res.loc[-1] = [sp[2] * sse.Em[2, socx] for socx in range(20)]

# initialize
# sse = ev_ssm(N=50000, tnow=10, name='SSE1')
# sse.ev[['u', 'u0', 'ctrl', 'xl']]
# sse.run(t_step=20, t_end=10.01, is_charge=False, is_record=True)
# sse.ev = sse.ev[0:100]
# sse.g_xl()


Run the ev model to get data

In [2]:
t_end = 10.1

sse = ev_ssm(N=10000, tnow=10, name='SSE1')

sse.run(t_step=20, t_end=t_end, is_charge=False, is_record=True)

SSE1:
tnow=10 [24H]
10000 EVs, 1927 online, Total En=250.23 MWh, SoC intervals: 20
Online En=48.26 MWh, mean SoC=63.0%
SSE1: 100%|██████████| 18/18 [00:07<00:00,  2.30it/s]


In [3]:
sse.g_A()

In [4]:
sse.Tm

array([[0.60345038, 0.15908901, 0.23746062],
       [0.05009646, 0.36442857, 0.58547497],
       [0.04327584, 0.33727159, 0.61945257]])

In [5]:
sse.Tf

array([[0.89898307, 0.        , 0.10101693],
       [0.        , 1.        , 0.        ],
       [0.8989547 , 0.        , 0.1010453 ]])

In [ ]:
sse.xtab

In [9]:
sse.xtab.sum(axis=1)/sse.xtab.sum().sum() [0]

IndexError: invalid index to scalar variable.

In [14]:
sse.Em[0, 19]

0.674391602908416

In [18]:
sp

 1    0.037606
 0    0.861758
-1    0.100636
dtype: float64

In [26]:
sp

 1    0.037606
 0    0.861758
-1    0.100636
dtype: float64

In [28]:
[sum(sp[0]*sse.Tm[0, 0] + sp[1]*sse.Tm[0, 1] + sp[2]*sse.Tm[0, 2]) * sse.Em[0, socx] for socx in range(20)]

0.03760593220338983

In [33]:
x = [sum([sp[i]*sse.Tm[0, i] for i in range(2)]) * sse.Em[0, socx] for socx in range(20)]
sum(x)

0.5260111655579601

In [38]:
sse.Em.round(3)

array([[0.   , 0.   , 0.   , 0.   , 0.   , 0.009, 0.038, 0.05 , 0.058,
        0.017, 0.025, 0.027, 0.028, 0.028, 0.015, 0.002, 0.01 , 0.015,
        0.005, 0.674],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.999],
       [0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.001, 0.   , 0.   ,
        0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
        0.   , 0.999]])

In [36]:
# state percentage
sp = sse.xtab.sum(axis=1)/sse.xtab.sum().sum()

out = pd.DataFrame(columns=range(20), index=[1, 0, -1])
out.loc[1] = [sum([sp[i]*sse.Tm[0, i] for i in range(2)]) * sse.Em[0, socx] for socx in range(20)]
out.loc[0] = [sum([sp[i]*sse.Tm[1, i] for i in range(2)]) * sse.Em[1, socx] for socx in range(20)]
out.loc[-1] = [sum([sp[i]*sse.Tm[2, i] for i in range(2)]) * sse.Em[2, socx] for socx in range(20)]

out.round(3).sum().sum()

0.6328636254192996

In [30]:
# state percentage
sp = sse.xtab.sum(axis=1)/sse.xtab.sum().sum()

out = pd.DataFrame(columns=range(20), index=[1, 0, -1])
out.loc[1] = [sp[1] * sse.Em[0, socx] for socx in range(20)]
out.loc[0] = [sp[0] * sse.Em[1, socx] for socx in range(20)]
out.loc[-1] = [sp[-1] * sse.Em[2, socx] for socx in range(20)]

out.round(3).sum().sum()

0.9999999999999999

In [35]:
out.sum(axis=1)

 1    0.037606
 0    0.861758
-1    0.100636
dtype: float64

In [23]:
out.round(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
1,0.0,0.0,0.0,0.0,0.0,0.0,0.001,0.002,0.002,0.001,0.001,0.001,0.001,0.001,0.001,0.0,0.0,0.001,0.0,0.025
2,0.0,0.0,0.0,0.0,0.0,0.0,0.001,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.0,0.861
3,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.000,0.0,0.101


In [ ]:
sse.ne

Calculate Mat. A, with interval 20s. from 10 to 11.

In [ ]:
sse.x

In [ ]:
# xres = pd.DataFrame()

# # calc to 10.1
# ss1 = ev_ssm(N=50000, tnow=10, name='ss1, calc')

# ss1.ev['soci'] = np.random.uniform(
#     low=0.0001,
#     high=0.9999,
#     size=sse.N)
# ss1.ev['socd'] = np.random.uniform(
#     low=0.0001,
#     high=0.9999,
#     size=sse.N)

# x1 = list(ss1.x.values())
# x1 = np.array(x1)
# for i in range(int(0.01*3600/20)):
#     x1 = np.matmul(A, x1)
# xres['calc0'] = list(ss1.x.values())
# xres['calc'] = x1

# # run to 10.1
# ss2 =  ev_ssm(N=50000, tnow=10, name='ss2, run')
# ss2.ev['soci'] = np.random.uniform(
#     low=0.0001,
#     high=0.9999,
#     size=sse.N)
# ss2.ev['socd'] = np.random.uniform(
#     low=0.0001,
#     high=0.9999,
#     size=sse.N)

# xres['run0'] = list(ss2.x.values())
# ss2.run(t_step=20, t_end=10.01, is_charge=True, is_record=False)
# xres['run'] = list(ss2.x.values())

# xres.round(4)

In [ ]:
# np.matmul(A.loc[39, :].values, list(ss1.x.values()))

In [ ]:
# ma = np.array([[0.2, 0.3], [0.8, 0.7]])
# mx = np.array([[50], [50]])
# mx1 = mx
# for i in range(100):
#     mx1 = np.matmul(ma, mx1)
# mx1

In [ ]:
# --- plot online EV numbers ---
time = np.arange(0, 24, 0.1)
num = []
for t in time:
    sse.g_u(t)
    num.append(sse.ne)

fig, ax = plt.subplots()
ax.plot(time, num)
ax.set_title('Number of online EVs')
ax.set_xlabel('Time [h]')
ax.legend(['Online EV', 'Online capacity'])
ax.set_xlim(0, 24)

In [ ]:
# # --- plot online EV numbers ---
# # time = np.arange(0, 24, 0.1)
# # num = []
# # for t in time:
# #     sse.g_u(t)
# #     num.append(sse.ne)

# # fig, ax = plt.subplots()
# # ax.plot(time, num)
# # ax.set_title('Number of online EVs')
# # ax.set_xlabel('Time [h]')
# # ax.legend(['Online EV', 'Online capacity'])
# # ax.set_xlim(0, 24)

# # update g_x()

# # # reset sse
# # sse.reset(tnow=10)
# # sse.ev

# # --- Analytical method ---
# data = sse.ev.copy()
# data['dsc'] = data.Pc * data.nc / data.En / 6
# data['dsd'] = data.Pc * data.nc / data.En / 1
# data[['dsc', 'dsd']].plot(kind='kde')

# kde = stats.gaussian_kde(data.dsc)
# for i in range(-1,18,1):
#     lb = 0.05+0.05*i
#     ub = 0.1+0.05*i
#     ires = kde.integrate_box(lb, ub)
#     # print(np.round(ires, 4))

In [ ]:
# xdf = pd.DataFrame(xres)
# from hmmlearn import hmm
# gen_model = hmm.GaussianHMM(n_components=60, covariance_type="full")


# sse.reset(tnow=0)
# xres = []
# xres.append(sse.x.values())
# for t in np.arange(0, 24, 300/3600):
#     sse.act(t=300, tnow=t)
#     xres.append(sse.x.values())
# xdf = pd.DataFrame(xres)


# gen_model.fit(pd.concat([xdf]*100).reset_index(drop=True))
# gen_model.predict_proba(xdf.iloc[2].values.reshape(1, -1))

## SSM Matrix

In [ ]:
# get the x res by iterate EV from t=0 to t=14.01
# sse.reset(tnow=0)
# x_res = []
# for i, t in enumerate(np.arange(0, 14.01, 1/12)):
#     sse.act(t=300, tnow=t)
#     x_res.append(sse.x.tolist())

# pd.DataFrame(x_res).plot(legend=True)

## Simulation

Issues: how to integerate the SSM when using ADNES?

flow_chart:
```{python}
prep grid data:
ADNES: topology,  gen. limits, ramp. limits, line limits,
Outside: gen. cost, ramp. cost,

for $t_{OPF}$ in T (interval: 5min; total: 1h; [n=12]):
    aggregate EV data (from SSM), generate $PR_{e,i,u,t}$
    Do OPF, generate $PG_{i, t}$, $PR_{g, i, u, t}$, $PR_{g, i, d, t}$

    for t in $t_{OPF}$ (interval: 4s; total: 5min; [n=75]):
        Update data into dynamic system:
            # Note, constant power model should be used in TDS.
            # Use TimeSeries as the load. 
            power change: TGOV1.paux0
            load change: 

        Run TDS: generate SFR mileage
```

Co-Sim list:
```{python}
for $t_{OPF}$ in T:
    EVA report $pru_{max}$ $prd_{max}$, eqn xxx
    TCC do OPF, eqn xxx
    Assign dispatch signal to generation units

    for t in $t_{AGC}$:
        Assign AGC signal to AGC units
        Run TDS
```